In [1]:
import os
import cv2
import configparser
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
print(tf.__version__)

2023-11-17 10:04:29.455584: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


2.3.0


In [2]:
root_path = "./translations"

In [3]:
train_path=root_path + "/mnist2mnist"
# train_path=root_path + "/mnist2svhn"
test_path=root_path + "/svhn2svhn"

In [4]:
datagen = ImageDataGenerator(rescale = 1./255)

In [5]:
train_gen = datagen.flow_from_directory(train_path,
                                        target_size = (32, 32),
                                        batch_size = 64,
                                        shuffle = True,
                                        class_mode = 'sparse')
test_gen = datagen.flow_from_directory(test_path,
                                       target_size = (32, 32),
                                       batch_size = 1,
                                       shuffle = False,
                                       class_mode = 'sparse')

Found 60000 images belonging to 10 classes.
Found 73257 images belonging to 10 classes.


In [6]:
train_y=train_gen.classes
test_y=test_gen.classes

In [7]:
from tensorflow.keras import layers
from tensorflow.keras.applications.resnet import ResNet50

In [8]:
IMG_SIZE=32
NUM_CLASSES=len(train_gen.class_indices)

In [9]:
# ResNet
def build_model(num_classes, img_size):
    
    resnet = ResNet50(input_shape=(img_size, img_size, 3), weights='imagenet', include_top=False)
    
    #do not train the pre-trained layers of ResNet50
    for layer in resnet.layers:
        layer.trainable = False
        
    x = layers.GlobalAveragePooling2D()(resnet.output)
    prediction = layers.Dense(num_classes, activation='softmax', name="pred")(x)

    model = tf.keras.Model(inputs=resnet.input, outputs=prediction, name="ResNet")

    return model

In [10]:
model = build_model(num_classes=NUM_CLASSES,img_size=IMG_SIZE)
print(model.summary())

2023-11-17 09:59:25.683635: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2023-11-17 09:59:25.845528: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:12:00.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.56GiB deviceMemoryBandwidth: 298.08GiB/s
2023-11-17 09:59:25.845572: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2023-11-17 09:59:25.927351: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2023-11-17 09:59:25.940114: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2023-11-17 09:59:25.958138: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcurand.so.10
2023-11-

Model: "ResNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
_____________________________________________________________________________________________

In [11]:
batch_size = 64
loss = 'sparse_categorical_crossentropy'
num_epochs = 100
optim = "adam"

model.compile(
      loss=loss,
      optimizer=optim,
      metrics=['sparse_categorical_accuracy']
    )

In [7]:
# Train on MNIST
model.fit(
    train_gen,
    steps_per_epoch = len(train_y)//batch_size,
    epochs = num_epochs,
)
model.save('./resnet50_mnist_svhn_model.hdf5')
model.evaluate(test_gen)

In [12]:
# Train on MNIST-to-SVHN
model.fit(
    train_gen,
    steps_per_epoch = len(train_y)//batch_size,
    epochs = num_epochs
)
model.save('./resnet50_mnist2svhn_svhn_model.hdf5')
model.evaluate(test_gen)

Epoch 1/100


2023-11-16 23:20:48.128387: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2023-11-16 23:20:48.968611: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7


937/937 [==============================] - 43s 46ms/step - loss: 1.9663 - sparse_categorical_accuracy: 0.3447
Epoch 2/100
937/937 [==============================] - 28s 30ms/step - loss: 1.5971 - sparse_categorical_accuracy: 0.5053
Epoch 3/100
937/937 [==============================] - 27s 29ms/step - loss: 1.4190 - sparse_categorical_accuracy: 0.5676
Epoch 4/100
937/937 [==============================] - 27s 29ms/step - loss: 1.3070 - sparse_categorical_accuracy: 0.6044
Epoch 5/100
937/937 [==============================] - 27s 29ms/step - loss: 1.2235 - sparse_categorical_accuracy: 0.6321
Epoch 6/100
937/937 [==============================] - 27s 29ms/step - loss: 1.1593 - sparse_categorical_accuracy: 0.6488
Epoch 7/100
937/937 [==============================] - 27s 28ms/step - loss: 1.1075 - sparse_categorical_accuracy: 0.6670
Epoch 8/100
937/937 [==============================] - 27s 29ms/step - loss: 1.0660 - sparse_categorical_accuracy: 0.6799
Epoch 9/100
937/937 [===============

[5.663160800933838, 0.18938805162906647]

In [8]:
hdf5_model = "./resnet50_mnist_svhn_model.hdf5"
loaded_model_from_hdf5 = tf.keras.models.load_model(hdf5_model)
loaded_model_from_hdf5.evaluate(test_gen)

2023-11-17 10:06:33.854424: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2023-11-17 10:06:34.021766: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:12:00.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.56GiB deviceMemoryBandwidth: 298.08GiB/s
2023-11-17 10:06:34.021809: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2023-11-17 10:06:34.110219: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2023-11-17 10:06:34.141606: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2023-11-17 10:06:34.160513: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcurand.so.10
2023-11-

73257/73257 [==============================] - 441s 6ms/step - loss: 7.2765 - sparse_categorical_accuracy: 0.1639


[7.276543617248535, 0.16392973065376282]